## Gina Martinez Lopez

# Character-Level LSTM in PyTorch

Construiremos una red neuronal recurrente tipo LSTM en PyTorch. La red entrenará caracter por caracter en un texto, luego generará texto nuevo caracter por caracter. Entrenamremos con un texto de un libro o novela que usted haya leído. Puede usar, por ejemplo [ESTE](https://www.gutenberg.org/) recurso para buscar un texto en archivo .txt. El texto generado estará basado en dicho libro.


Referencias: Andrej Karpathy [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) y [implementation in Torch](https://github.com/karpathy/char-rnn).

El ejercicio consiste en realizar las instrucciones marcadas como TO_DO. Entregar a más tardar el 30 de marzo.


Importamos las librerías y paquetes requeridos.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Cargar los datos

Cargamos el texto en un archivo .txt y lo convertimos en enteros para que la red lo use.

In [2]:
# open text file and read in data as `text`
with open('/content/AdanyEva.txt', 'r') as f:
    text = f.read()

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Imprimimos los primeros 100 characteres y revisamos que el texto luzca bien.

In [4]:
text[:100]

'Adán, Padre de los Hombres, fue creado en el día 28 de octubre, a las\ndos de la tarde... Afírmalo as'

### Tokenización

En las celdas siguientes, creamos un par de diccionarios para convertir los caracteres desde y hacia enteros. Esto los vuelve más fáciles de usar como entrada de la red.

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Imprimamos los caracteres anteriores, pero ahora codificados como enteros.

In [6]:
encoded[:100]

array([ 37,  76,  17,  71, 107,  72,  57,  62,  76,  49,   0,  72,  76,
         0,  72,  38,  77,  90,  72,  92,  77,  96,  18,  49,   0,  90,
       107,  72,  58,  39,   0,  72,  33,  49,   0,  62,  76,  77,  72,
         0,  71,  72,   0,  38,  72,  76,   5,  62,  72,  30,  23,  72,
        76,   0,  72,  77,  33,  91,  39,  18,  49,   0, 107,  72,  62,
        72,  38,  62,  90,  25,  76,  77,  90,  72,  76,   0,  72,  38,
        62,  72,  91,  62,  49,  76,   0,  60,  60,  60,  72,  37,  58,
         5,  49,  96,  62,  38,  77,  72,  62,  90])

## Pre-procesamiento de los datos

Nuestra red LSTM espera una entrada codificada con **one hot encoding**. Definimos una función para esto.


In [7]:
def one_hot_encode(arr, n_labels):

    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Crear mini-batches de entrenamiento

Para entrenar en este texto, queremos crear mini lotes.

En este ejemplo, tomaremos los caracteres codificados (los pasamos como parámetro `arr`) y los separaremos en múltiples trozos o sequencias. A saber, el *número de trozos* que se ingresa a la red es `batch_size` (tenga en cuenta que no es equivalente al batchsize que usábamos antes). Cada sequencia ingresada a la red tendrá tamaño `seq_length` (éste sí es equivalente al batch size anterior).

### Creando los lotes

**Lo primero que debemos hacer es descartar algún texto, de manera que sólo tengamos mini lotes de tamaño completo.**

Cada lote contiene $N \times M$ caracteres, en donde $N$ es el número de secuencias en cada lote, y $M$ es seq_length o el número de pasos de tiempo en una secuencia. Luego, para obtener el número total $K$ de lotes, podemos dividir la longitud de `arr`, entre el número de caracteres por lote. El número total de caracteres que usaremos del texto sería entonces $N * M * K$. Lo que sobre, deberíamos descartarlo.

**Luego de eso, necesitamos separar todo el texto que usaremos `arr` en $N$ trozos.**

Puede hacer esto usando `arr.reshape(size)`, en donde `size` es una tupla con las dimensiones del arreglo re-dimensionado. Sabemos que queremos $N$ trozos (líneas) con todo el texto, entonces que esa sea la primera dimensión. Para la segunda dimensión podemos escribir directamente su tamaño, o usar el valor `-1`. Éste llenará el arreglo con el tamaño apropiado, dependiento de la cantidad de entradas disponibles.

**3. Ahora que tenemos este arreglo, podemos iterar a través de él para obtener nuestos lotes con los cuales alimentar la red.**

La idea es que cada lote sea una ventana de tamaño $N \times M$, del gran arreglo de tamaño $N \times (M * K)$. Para cada lote siguiente, la ventana se mueve una distancia de `seq_length`.

Adicionalmente queremos crear arreglos de entrada y arreglos de etiquetas.Recuerde que las etiquetas són las mismas entradas pero desfasadas por un caracter.


> **TODO:** Escriba el código para crear lotes en la función de abajo.

In [9]:
import math
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    #print(f"arr.shape: {arr.shape}")
    ## TO_DO: Calcular el número de lotes que se pueden obter del texto. Llamarlo n_batches.
    n_batches = len(arr) // (batch_size * seq_length)
    #n_batches = arr.shape[1] // (batch_size * seq_length)

    ## TO_DO: Conservar una cantidad exacta de caracteres, de tal forma que sólo haya lotes completos.
    arr = arr[:n_batches * batch_size * seq_length]

    ## TO_DO: Redimensionar arr de tal forma que quede de tamaño batch_size*(lo que quede)
    arr = arr.reshape((batch_size, -1))

    ## Iterar sobre los lotes, usando una ventana de tamaño seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:,n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Compruebe su implementación

Crearemos algunos lotes para revisar lo que ocurre. Usaremos batch_size de 8 y tamaño de la secuencia 50.

In [10]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 37  76  17  71 107  72  57  62  76  49]
 [ 74  77  49  62  72  62  72  76   0  71]
 [ 49  91   0  72  65  72   0  90  72  18]
 [  0  60  72   4  62  72  96  34  90   0]
 [ 72  11  62  38  74  62  91  34   0  49]
 [  0  91  34  21  72   0  38  72  70   0]
 [ 72  91   0  96  70  49  62  71  77 107]
 [ 91  17  72   0  38  72  64  62  33  64]]

y
 [[ 76  17  71 107  72  57  62  76  49   0]
 [ 77  49  62  72  62  72  76   0  71  91]
 [ 91   0  72  65  72   0  90  72  18   0]
 [ 60  72   4  62  72  96  34  90   0  49]
 [ 11  62  38  74  62  91  34   0  49  49]
 [ 91  34  21  72   0  38  72  70   0  49]
 [ 91   0  96  70  49  62  71  77 107  72]
 [ 17  72   0  38  72  64  62  33  64  62]]


Si se implementó `get_batches` correctamente, la salida debería verse similar a esta:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
Los números pueden ser diferentes. Asegúrese de que los valores en y están desfasados en 1.

---
## Definir la red con Pytorch

A continuación definiremos la red. Usamos Pytorch para definir la arquitectura de la red. Comenzamos definiendo el forward pass. Se provee también un método para predecir caracteres.

### Esctructura del modelo

En `__init__` la estructura sugerida es como sigue:
* Crear y guardar los diccionarios requeridos (diccionarios para codificar caracteres)
* Definir una capa LSTM que tome como parámetros: un tamaño de entrada (el número de caracteres), el tamaño de la capa oculta `n_hidden`, el número de capas `n_layers`, una probabilidad para Dropout `drop_prob`.
* Definir una capa Dropout con `drop_prob`
* Definir una capa lineal con parámetros: tamaño de entrada `n_hidden` y tamaño de salida (el número de caracteres)

Note que algunos parámetros se han nombrado y están dados en la función `__init__`, y los usamos y guardamos con una línea tipo `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

Se puede crear una capa [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) como sigue

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
```

en donde `input_size` es el número de características esperadas en la entrada en cada momento de tiempo, `n_hidden` es el tamaño de las capas ocultas y `n_layers` es el número de capas ocultas. Y podemos añadir un parámetro de Dropout con la probabilidad específica.

Necesitamos también inicializar los estados iniciales como ceros. (Eso está hecho ya)


```python
self.init_hidden()
```

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else:
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [13]:
class CharRNN(nn.Module):

    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}


        ## TO_DO: definir la capa LSTM.
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, batch_first=True, dropout=drop_prob)

        ## TO_DO: definir una capa de Dropout
        self.dropout = nn.Dropout(drop_prob)


        ## TO_DO: Definir una capa final, la cual es fully-connected, cuya entrada es de tamaño igual al tamaño de las capas ocultas,
        # y la salida es en este caso de mismo tamaño de la entrada (un vector con una entrada para cada caracter)

        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        ''' Forward pass through the network.
            These inputs are x, and the hidden/cell state `hidden`. '''

        batch_size = x.size(0)

        # get RNN outputs
        r_out, hidden = self.lstm(x, hidden)
        # shape output to be (batch_size*seq_length, hidden_dim)
        r_out = r_out.reshape(-1, self.n_hidden)

        r_out=self.dropout(r_out)
        # get final output
        output = self.fc(r_out)


        # return the final output and the hidden state
        return output, hidden


    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

        return hidden


## Entrenamiento de la red




In [14]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())

                    val_losses.append(val_loss.item())

                net.train() # reset to train mode after iterationg through validation data

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Inicializamos el modelo

Ahora podemos entrenar la red. Primero establecemos los hiperparámetros. Luego definimos las dimensiones de los lotes (batches) e iniciamos el entrenamiento.


In [15]:
## TO_DO: Establecer los hiperparámetros para su modelo
# define and print the net
n_hidden=256*2
n_layers=3

## Definir e imprimir net
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(111, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=111, bias=True)
)


## Definir hiperparámetros de entrenamiento y entrenar

In [16]:
#TO_DO definir hiperparámetros del entrenamiento
batch_size = 64
seq_length = 128
n_epochs = 100 # empezar con un número pequeño al inicio para verificar el comportamiento del algoritmo

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/100... Step: 10... Loss: 3.1995... Val Loss: 3.2910
Epoch: 1/100... Step: 20... Loss: 3.1433... Val Loss: 3.2694
Epoch: 1/100... Step: 30... Loss: 3.0943... Val Loss: 3.2510
Epoch: 1/100... Step: 40... Loss: 3.1229... Val Loss: 3.2493
Epoch: 2/100... Step: 50... Loss: 3.1223... Val Loss: 3.2506
Epoch: 2/100... Step: 60... Loss: 3.1052... Val Loss: 3.2488
Epoch: 2/100... Step: 70... Loss: 3.1143... Val Loss: 3.2530
Epoch: 2/100... Step: 80... Loss: 3.1116... Val Loss: 3.2599
Epoch: 2/100... Step: 90... Loss: 3.0703... Val Loss: 3.2577
Epoch: 3/100... Step: 100... Loss: 3.0411... Val Loss: 3.2502
Epoch: 3/100... Step: 110... Loss: 3.0939... Val Loss: 3.2429
Epoch: 3/100... Step: 120... Loss: 3.0458... Val Loss: 3.2602
Epoch: 3/100... Step: 130... Loss: 2.9929... Val Loss: 3.1762
Epoch: 3/100... Step: 140... Loss: 2.7993... Val Loss: 3.1181
Epoch: 4/100... Step: 150... Loss: 2.7456... Val Loss: 3.1004
Epoch: 4/100... Step: 160... Loss: 2.6921... Val Loss: 3.1061
Epoch: 4/100... S

# Parte 2

## Guardar el modelo

Luego de entrenar, vamos a guardar el modelo de manera que podamos cargarlo luego si lo necesitamos.
A continuación, veamos la arquitectura del modelo y luego de eso lo guardamos:

In [17]:
net.state_dict()

OrderedDict([('lstm.weight_ih_l0',
              tensor([[-0.0856,  0.0433, -0.0042,  ...,  0.5023,  0.0773,  0.0246],
                      [-0.0705, -0.0385,  0.0328,  ..., -0.1021, -0.1255,  0.1607],
                      [ 0.0843, -0.0299, -0.0303,  ...,  0.0956,  0.1725,  0.4327],
                      ...,
                      [-0.2728,  0.0023, -0.0028,  ..., -0.2996,  0.0974,  0.0850],
                      [-0.2461,  0.0114, -0.0012,  ..., -0.1068, -0.1032,  0.1914],
                      [ 0.3577,  0.0195, -0.0333,  ..., -0.0178,  0.0283, -0.0805]],
                     device='cuda:0')),
             ('lstm.weight_hh_l0',
              tensor([[ 0.0965, -0.0679,  0.0131,  ..., -0.0068, -0.1385,  0.1106],
                      [ 0.0557, -0.0522,  0.0799,  ...,  0.1387, -0.1215,  0.0583],
                      [ 0.0517,  0.0506, -0.0236,  ..., -0.0551, -0.0771, -0.0836],
                      ...,
                      [ 0.2478,  0.0369,  0.0353,  ..., -0.0303,  0.0778,  0.08

In [18]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Haciendo predicciones

Ahora que el modelo está entrenado, queremos usarlo para hacer predicciones sobre los caracteres siguientes. Para obtener una muestra de de texto,  le pasamos un caracter y dejamos que la red prediga el siguiente caracter. Luego tomamos dicho caracter, lo pasamos a la red y obtenemos el siguiente caracter predicho. Seguimos haciendo lo mismo hasta generar una porción de texto.


### Observación

La salida de nuestra red neuronal viene de una capa lineal (fully-connected) y produce como salida una distribución de puntajes para el siguiente caracter.

Para obtener el caracter siguiente, aplicamos una función softmax, que nos da una distribución de probabilidad.

### Top K (los $K$ más probables)
Nuestras predicciones vienen de una distribución de probabilidad sobre todos los caracteres posibles. Podemos generar las predicciones considerando solamente los $K$ caracteres más probables. Esto prevendrá a la red de arrojar caracteres absurdos, pero permitiendo introducir algo de aleatoriedad en el texto generado.



In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''

        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)

        if(train_on_gpu):
            inputs = inputs.cuda()

        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())

        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Preparar la red y generar el texto

Típicamente se querrá preparar la red con unos primeros caracteres (estos los definimos en el parámetro 'prime'), de manera que se pueda construir un primer estado $h_0$.  De lo contrario la red comenzará generando caracteres aleatoriamente.

In [20]:
def sample(net, size, prime='The', top_k=None):

    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()

    net.eval() # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [21]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annacen), a pesar de casa de los cocodrilosos, que aprovechando el aroma del pobre Materra. El pobre Guannes, contempla
el callareo escarrritando, de seguro en la punta
de una antigua, que eran desnudos, escurriendo las pulidras y las
heridas de arrependible al monte de un color que le dure estas manas en en
campo más presamente al altar. Para su celestial madrina, el medio de aquella torde de su vida; y el mundo cubierta de amarle es un corrón, que duró, pasaba el mármol que esparcía.

El pueblo se acodaba deserado de este sol, casi levantada y rico, sin cabellos blancos; de cadera, aparecía a los cuales descomenzando por la puerta de los
cabellos, a lo lindo de la palma; a los cuales era un profundo cuatro por la cabeza del mostrador. A poco,
como en esa semana como en su pasado
y solitario
que le
había sido los ojos y de lentaje, en
el comedor de medios, de contigua friolesta si toda le encaje... Y sin comprenderle, y caminó, para su
comprender don Alino. En esto,
conservaba los des

### Conclusión:

**TO_DO** ¿Qué libro eligió? EL texto generado tiene sentido gramaticalmente? ¿El estilo del texto es similar al de su libro? ¿Por qué?  Si alguna de estas respuestas es negativa, considere diferentes hiperparámetros para su modelo.

Elegí Adan y Eva en español, gramaticalmente se ve bien, el estilo es similar,es similar porque estamos trabajando sobre el texto y hace una predicción sobre este. Sin embargo, no sé que tanta coherencia tenga
(Dado es caracter por caracter es dificil obtener coherencia).